# Chord

In [ ]:
# IMPORTS

import os
import time

import numpy as np

import madmom

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset as Dataset
import torch.optim as optim

from sklearn.metrics import accuracy_score, precision_score, recall_score

# configurations
import scripts.chord_config as cc

# feature, target, annotation initializer
from scripts.chord_feat import init_data

from scripts.chord_util import parse_annotations

In [ ]:
# LOAD FEATURES AND ANNOTATIONS, COMPUTE TARGETS
train_f, train_t, train_anno, valid_f, valid_t, valid_anno, test_f, test_t, test_anno = init_data()

In [ ]:
valid_t[0]